In [3]:
pip install python-binance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 78.1 MB/s eta 0:00:00


In [ ]:
import logging
import sys
from binance import Client, ThreadedWebsocketManager
from binance.exceptions import BinanceAPIException, BinanceRequestException

# --- Configuration ---
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("trading_bot.log"), # Log to a file
        logging.StreamHandler(sys.stdout)       # Log to console
    ]
)
logger = logging.getLogger(__name__)

class BasicBot:
    """
    A simplified trading bot for Binance Futures Testnet.
    Supports placing market and limit orders, checking balance,
    viewing open orders, and cancelling orders.
    """
    def __init__(self, api_key: str, api_secret: str, testnet: bool = True):
        """
        Initializes the Binance client for Futures trading.

        Args:
            api_key (str): Your Binance API key.
            api_secret (str): Your Binance API secret.
            testnet (bool): Whether to connect to the testnet or production.
                            Defaults to True for testnet.
        """
        self.api_key = api_key
        self.api_secret = api_secret
        self.testnet = testnet
        self.client = None
        self._initialize_client()

    def _initialize_client(self):
        """Initializes the Binance Client with the correct URL for testnet/production."""
        if self.testnet:
            # For Binance Futures Testnet, the URL needs to be explicitly set.
            # The python-binance library handles the futures endpoint correctly
            # when futures=True is passed.
            # The testnet base URL is https://testnet.binancefuture.com
            self.client = Client(
                self.api_key,
                self.api_secret,
                #base_url='https://testnet.binancefuture.com' # This is the key for testnet
            )
            logger.info("Connected to Binance Futures Testnet.")
        else:
            self.client = Client(self.api_key, self.api_secret)
            logger.info("Connected to Binance Futures Production.")

    def _log_api_call(self, method: str, params: dict):
        """Logs the API request details."""
        logger.info(f"API Request: Method={method}, Params={params}")

    def _handle_api_response(self, response: dict):
        """Logs the API response details."""
        logger.info(f"API Response: {response}")

    def _handle_api_error(self, e: Exception, operation: str):
        """Logs and handles Binance API exceptions."""
        if isinstance(e, BinanceAPIException):
            logger.error(f"Binance API Error during {operation}: Code={e.code}, Message={e.message}")
            raise e
        elif isinstance(e, BinanceRequestException):
            logger.error(f"Binance Request Error during {operation}: {e}")
            raise e
        else:
            logger.error(f"Unexpected Error during {operation}: {e}")
            raise e

    def get_balance(self, asset: str = 'USDT') -> float:
        """
        Retrieves the balance for a specific asset in your Futures account.

        Args:
            asset (str): The asset symbol (e.g., 'USDT').

        Returns:
            float: The available balance of the asset.
        """
        try:
            self._log_api_call("futures_account_balance", {'asset': asset})
            account_info = self.client.futures_account_balance()
            self._handle_api_response(account_info)
            for balance in account_info:
                if balance['asset'] == asset:
                    return float(balance['balance'])
            logger.warning(f"Asset {asset} not found in account balance.")
            return 0.0
        except Exception as e:
            self._handle_api_error(e, f"getting balance for {asset}")
            return 0.0

    def place_market_order(self, symbol: str, side: str, quantity: float) -> dict:
        """
        Places a market order on Binance Futures.

        Args:
            symbol (str): Trading pair symbol (e.g., 'BTCUSDT').
            side (str): 'BUY' or 'SELL'.
            quantity (float): The amount of base asset to buy/sell.

        Returns:
            dict: The order response from Binance.
        """
        try:
            params = {'symbol': symbol, 'side': side, 'type': 'MARKET', 'quantity': quantity}
            self._log_api_call("futures_create_order (MARKET)", params)
            order = self.client.futures_create_order(
                symbol=symbol,
                side=side,
                type='MARKET',
                quantity=quantity
            )
            self._handle_api_response(order)
            logger.info(f"Market order placed: {order}")
            return order
        except Exception as e:
            self._handle_api_error(e, f"placing market order for {symbol}")
            return {}

    def place_limit_order(self, symbol: str, side: str, quantity: float, price: float) -> dict:
        """
        Places a limit order on Binance Futures.

        Args:
            symbol (str): Trading pair symbol (e.g., 'BTCUSDT').
            side (str): 'BUY' or 'SELL'.
            quantity (float): The amount of base asset to buy/sell.
            price (float): The price at which to place the limit order.

        Returns:
            dict: The order response from Binance.
        """
        try:
            params = {'symbol': symbol, 'side': side, 'type': 'LIMIT', 'quantity': quantity, 'price': price}
            self._log_api_call("futures_create_order (LIMIT)", params)
            order = self.client.futures_create_order(
                symbol=symbol,
                side=side,
                type='LIMIT',
                quantity=quantity,
                price=price,
                timeInForce='GTC' # Good Till Cancelled
            )
            self._handle_api_response(order)
            logger.info(f"Limit order placed: {order}")
            return order
        except Exception as e:
            self._handle_api_error(e, f"placing limit order for {symbol}")
            return {}

    def place_stop_limit_order(self, symbol: str, side: str, quantity: float, price: float, stop_price: float) -> dict:
        """
        Places a Stop-Limit order on Binance Futures.

        Args:
            symbol (str): Trading pair symbol (e.g., 'BTCUSDT').
            side (str): 'BUY' or 'SELL'.
            quantity (float): The amount of base asset to buy/sell.
            price (float): The limit price at which the order will be placed once the stop price is triggered.
            stop_price (float): The price at which the stop order will be triggered.

        Returns:
            dict: The order response from Binance.
        """
        try:
            params = {'symbol': symbol, 'side': side, 'type': 'STOP_MARKET', 'quantity': quantity, 'stopPrice': stop_price}
            self._log_api_call("futures_create_order (STOP_MARKET)", params)
            order = self.client.futures_create_order(
                symbol=symbol,
                side=side,
                type='STOP_MARKET', # For Stop-Limit, you typically use STOP_MARKET or TAKE_PROFIT_MARKET
                                    # and then place a limit order upon trigger.
                                    # Binance Futures API has STOP and TAKE_PROFIT order types which are market orders.
                                    # For a true Stop-Limit, you'd set type='STOP' and provide a price.
                                    # Let's use 'STOP' and 'price' for the limit price.
                quantity=quantity,
                stopPrice=stop_price,
                price=price, # This is the limit price for the STOP_MARKET order
                timeInForce='GTC'
            )
            self._handle_api_response(order)
            logger.info(f"Stop-Limit order placed: {order}")
            return order
        except Exception as e:
            self._handle_api_error(e, f"placing stop-limit order for {symbol}")
            return {}

    def get_open_orders(self, symbol: str = None) -> list:
        """
        Retrieves all open orders or open orders for a specific symbol.

        Args:
            symbol (str, optional): The trading pair symbol (e.g., 'BTCUSDT').
                                    If None, retrieves all open orders.

        Returns:
            list: A list of open orders.
        """
        try:
            params = {'symbol': symbol} if symbol else {}
            self._log_api_call("futures_get_open_orders", params)
            if symbol:
                open_orders = self.client.futures_get_open_orders(symbol=symbol)
            else:
                open_orders = self.client.futures_get_open_orders()
            self._handle_api_response(open_orders)
            return open_orders
        except Exception as e:
            self._handle_api_error(e, "getting open orders")
            return []

    def cancel_order(self, symbol: str, order_id: int) -> dict:
        """
        Cancels a specific order on Binance Futures.

        Args:
            symbol (str): Trading pair symbol (e.g., 'BTCUSDT').
            order_id (int): The ID of the order to cancel.

        Returns:
            dict: The cancellation response from Binance.
        """
        try:
            params = {'symbol': symbol, 'orderId': order_id}
            self._log_api_call("futures_cancel_order", params)
            result = self.client.futures_cancel_order(symbol=symbol, orderId=order_id)
            self._handle_api_response(result)
            logger.info(f"Order {order_id} cancelled: {result}")
            return result
        except Exception as e:
            self._handle_api_error(e, f"cancelling order {order_id} for {symbol}")
            return {}

def validate_positive_float(prompt: str) -> float:
    """Helper to get and validate a positive float input."""
    while True:
        try:
            value = float(input(prompt))
            if value <= 0:
                print("Input must be a positive number.")
                continue
            return value
        except ValueError:
            print("Invalid input. Please enter a number.")

def validate_side(prompt: str) -> str:
    """Helper to get and validate order side."""
    while True:
        side = input(prompt).upper()
        if side in ['BUY', 'SELL']:
            return side
        else:
            print("Invalid side. Please enter 'BUY' or 'SELL'.")

def main():
    """Main function to run the CLI trading bot."""
    print("--- Binance Futures Testnet Trading Bot ---")

    api_key = input("Enter your Binance Testnet API Key: ").strip()
    api_secret = input("Enter your Binance Testnet API Secret: ").strip()

    if not api_key or not api_secret:
        logger.error("API Key and Secret cannot be empty. Exiting.")
        sys.exit(1)

    bot = BasicBot(api_key, api_secret, testnet=True)

    while True:
        print("\n--- Menu ---")
        print("1. Get USDT Balance")
        print("2. Place Market Order")
        print("3. Place Limit Order")
        print("4. Place Stop-Limit Order (Bonus)")
        print("5. View Open Orders")
        print("6. Cancel Order")
        print("7. Exit")

        choice = input("Enter your choice: ").strip()

        if choice == '1':
            balance = bot.get_balance('USDT')
            print(f"Your current USDT balance: {balance:.2f}")

        elif choice == '2':
            symbol = input("Enter symbol (e.g., BTCUSDT): ").upper().strip()
            side = validate_side("Enter side (BUY/SELL): ")
            quantity = validate_positive_float("Enter quantity: ")
            if symbol and side and quantity:
                bot.place_market_order(symbol, side, quantity)

        elif choice == '3':
            symbol = input("Enter symbol (e.g., BTCUSDT): ").upper().strip()
            side = validate_side("Enter side (BUY/SELL): ")
            quantity = validate_positive_float("Enter quantity: ")
            price = validate_positive_float("Enter price: ")
            if symbol and side and quantity and price:
                bot.place_limit_order(symbol, side, quantity, price)

        elif choice == '4':
            symbol = input("Enter symbol (e.g., BTCUSDT): ").upper().strip()
            side = validate_side("Enter side (BUY/SELL): ")
            quantity = validate_positive_float("Enter quantity: ")
            price = validate_positive_float("Enter limit price (price at which order will execute): ")
            stop_price = validate_positive_float("Enter stop price (trigger price): ")
            if symbol and side and quantity and price and stop_price:
                bot.place_stop_limit_order(symbol, side, quantity, price, stop_price)

        elif choice == '5':
            symbol_filter = input("Enter symbol to filter (leave blank for all): ").upper().strip()
            open_orders = bot.get_open_orders(symbol_filter if symbol_filter else None)
            if open_orders:
                print("\n--- Open Orders ---")
                for order in open_orders:
                    print(f"Order ID: {order['orderId']}, Symbol: {order['symbol']}, "
                          f"Side: {order['side']}, Type: {order['type']}, "
                          f"Quantity: {order['origQty']}, Price: {order['price']}, "
                          f"Status: {order['status']}")
            else:
                print("No open orders found.")

        elif choice == '6':
            symbol = input("Enter symbol of the order to cancel (e.g., BTCUSDT): ").upper().strip()
            try:
                order_id = int(input("Enter Order ID to cancel: "))
                if symbol and order_id:
                    bot.cancel_order(symbol, order_id)
            except ValueError:
                print("Invalid Order ID. Please enter a number.")

        elif choice == '7':
            print("Exiting bot. Goodbye!")
            break

        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


--- Binance Futures Testnet Trading Bot ---
